# Data Preprocessing Workflow

By: Traci Lim

---


**The notebook shows the workflow on data preprocessing**. The final dataset will be shipped to Tableau for visualizations.

Several external datasets were either retrieved from OneMap API or downloaded from [data.gov.sg](https://data.gov.sg/).
- Batch-geocoded latitude and longitude data: ```latlong_df```; retrieved from [link](https://docs.onemap.sg/#onemap-rest-apis)
- MRT stations exits address data with geocodes: ```station_latlong_df```; retrieved from [link](https://data.gov.sg/dataset/sdcp-mrt-station-point?resource_id=1123aaa2-4575-4181-81f6-8e11cda8bd8a [D])
- Schools address data with geocodes: ```sch_distance_df```; retrieved from [link](https://data.gov.sg/dataset/school-directory-and-information)
- Extra Information on HDB address data: ```hdb_extra_df_final```; retrieved from [link](https://data.gov.sg/dataset/hdb-property-information)
- COE data: ```coe_df```; retrieved from [link](https://data.gov.sg/dataset/coe-bidding-results?resource_id=f7bbdc43-c568-4e60-9afa-b77ba5a14aa0)


*Note that this notebook is littered with short comments, and i address myself as 'we' here to keep some level of formality in this documentation.*

In [3]:
import glob
import os
import gc
import pandas as pd
import geopy.distance
pd.set_option('display.max_columns', None)  
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

def reduce_memory(df):
    for c in df.columns:
        if df[c].dtype=='int':
            if df[c].min()<0:
                if df[c].abs().max()<2**7:
                    df[c] = df[c].astype('int8')
                elif df[c].abs().max()<2**15:
                    df[c] = df[c].astype('int16')
                elif df[c].abs().max()<2**31:
                    df[c] = df[c].astype('int32')
                else:
                    continue
            else:
                if df[c].max()<2**8:
                    df[c] = df[c].astype('uint8')
                elif df[c].max()<2**16:
                    df[c] = df[c].astype('uint16')
                elif df[c].max()<2**32:
                    df[c] = df[c].astype('uint32')
                else:
                    continue
    return df

In [205]:
# A loop to display each dataset's schema 
for files in glob.glob("./data/hdb_resale/*.csv"):
    df = pd.read_csv(files)
    print (files, 'has ', len(df.columns), 'columns\n')
    print (df.columns, '\n')
    print (df.dtypes, '\n')

./data/hdb_resale\resale-flat-prices-based-on-approval-date-1990-1999.csv has  10 columns

Index(['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range',
       'floor_area_sqm', 'flat_model', 'lease_commence_date', 'resale_price'],
      dtype='object') 

month                   object
town                    object
flat_type               object
block                   object
street_name             object
storey_range            object
floor_area_sqm         float64
flat_model              object
lease_commence_date      int64
resale_price             int64
dtype: object 

./data/hdb_resale\resale-flat-prices-based-on-approval-date-1990-1999_1.csv has  10 columns

Index(['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range',
       'floor_area_sqm', 'flat_model', 'lease_commence_date', 'resale_price'],
      dtype='object') 

month                   object
town                    object
flat_type               object
block                   object
street

Turns out all 4 datasets that dates from 2015, ```resale-flat-prices-based-on-registration-date-from-jan-2015-onwards_i.csv``` have an extra column—```'remaining_lease'```. All columns in datasets looks to be in the correct type, which is good. We can proceed to concatenating all datasets into one dataframe.

In [282]:
df = pd.DataFrame()
for files in glob.glob("./data/hdb_resale/*.csv"):
    print ('Concatenating...', files)
    df = pd.concat([df,pd.read_csv(files).iloc[:,:]],axis=0)
print ('Datasets are now concatenated into one dataframe.')

Concatenating... ./data/hdb_resale\resale-flat-prices-based-on-approval-date-1990-1999.csv
Concatenating... ./data/hdb_resale\resale-flat-prices-based-on-approval-date-1990-1999_1.csv
Concatenating... ./data/hdb_resale\resale-flat-prices-based-on-approval-date-1990-1999_2.csv
Concatenating... ./data/hdb_resale\resale-flat-prices-based-on-approval-date-1990-1999_3.csv
Concatenating... ./data/hdb_resale\resale-flat-prices-based-on-approval-date-2000-feb-2012.csv
Concatenating... ./data/hdb_resale\resale-flat-prices-based-on-approval-date-2000-feb-2012_1.csv
Concatenating... ./data/hdb_resale\resale-flat-prices-based-on-approval-date-2000-feb-2012_2.csv
Concatenating... ./data/hdb_resale\resale-flat-prices-based-on-approval-date-2000-feb-2012_3.csv
Concatenating... ./data/hdb_resale\resale-flat-prices-based-on-registration-date-from-jan-2015-onwards.csv
Concatenating... ./data/hdb_resale\resale-flat-prices-based-on-registration-date-from-jan-2015-onwards_1.csv
Concatenating... ./data/hdb_

---

## Feeling the data out

In [283]:
df.shape

(3100996, 11)

In [208]:
df.head()

,block,flat_model,flat_type,floor_area_sqm,lease_commence_date,month,remaining_lease,resale_price,storey_range,street_name,town
0,309,IMPROVED,1 ROOM,31.0,1977,1990-01,NaN,9000.0,10 TO 12,ANG MO KIO AVE 1,ANG MO KIO
1,309,IMPROVED,1 ROOM,31.0,1977,1990-01,NaN,6000.0,04 TO 06,ANG MO KIO AVE 1,ANG MO KIO
2,309,IMPROVED,1 ROOM,31.0,1977,1990-01,NaN,8000.0,10 TO 12,ANG MO KIO AVE 1,ANG MO KIO
3,309,IMPROVED,1 ROOM,31.0,1977,1990-01,NaN,6000.0,07 TO 09,ANG MO KIO AVE 1,ANG MO KIO
4,216,NEW GENERATION,3 ROOM,73.0,1976,1990-01,NaN,47200.0,04 TO 06,ANG MO KIO AVE 1,ANG MO KIO


Notice that the column ```'remaining_lease'``` contains mostly NaNs. This feature does not ring any bells on how useful it can be, so we will delete it for the time being.

In [284]:
del df['remaining_lease']

In [243]:
df.dtypes

block                   object
flat_model              object
flat_type               object
floor_area_sqm         float64
lease_commence_date      int64
month                   object
resale_price           float64
storey_range            object
street_name             object
town                    object
dtype: object

Next, we concatenate ```block``` and ```street_name``` to get the full address for each row. This step is important because we want to convert these addresses into queries, so as to pass into the OneMap API.  

In [285]:
df["full_address"] = df["block"].map(str) + ' ' + df["street_name"].map(str)

In [286]:
df.head(3)

,block,flat_model,flat_type,floor_area_sqm,lease_commence_date,month,resale_price,storey_range,street_name,town,full_address
0,309,IMPROVED,1 ROOM,31.0,1977,1990-01,9000.0,10 TO 12,ANG MO KIO AVE 1,ANG MO KIO,309 ANG MO KIO AVE 1
1,309,IMPROVED,1 ROOM,31.0,1977,1990-01,6000.0,04 TO 06,ANG MO KIO AVE 1,ANG MO KIO,309 ANG MO KIO AVE 1
2,309,IMPROVED,1 ROOM,31.0,1977,1990-01,8000.0,10 TO 12,ANG MO KIO AVE 1,ANG MO KIO,309 ANG MO KIO AVE 1


In [80]:
# Get number of unique full addresses
full_address_list = df["full_address"].unique().tolist()
len(full_address_list)

8838

There are 8838 unique addresses in all property resale transactions data. 

In [81]:
full_address_list[1]

'216 ANG MO KIO AVE 1'

In [122]:
test_address_list = full_address_list[1:200]

In [123]:
test_address_list

['216 ANG MO KIO AVE 1',
 '211 ANG MO KIO AVE 3',
 '202 ANG MO KIO AVE 3',
 '235 ANG MO KIO AVE 3',
 '232 ANG MO KIO AVE 3',
 '308 ANG MO KIO AVE 1',
 '220 ANG MO KIO AVE 1',
 '219 ANG MO KIO AVE 1',
 '247 ANG MO KIO AVE 3',
 '320 ANG MO KIO AVE 1',
 '252 ANG MO KIO AVE 4',
 '223 ANG MO KIO AVE 1',
 '230 ANG MO KIO AVE 3',
 '329 ANG MO KIO AVE 3',
 '313 ANG MO KIO AVE 3',
 '117 ANG MO KIO AVE 4',
 '110 ANG MO KIO AVE 4',
 '343 ANG MO KIO AVE 3',
 '345 ANG MO KIO AVE 3',
 '346 ANG MO KIO AVE 3',
 '121 ANG MO KIO AVE 3',
 '129 ANG MO KIO AVE 3',
 '130 ANG MO KIO AVE 3',
 '128 ANG MO KIO AVE 3',
 '127 ANG MO KIO AVE 3',
 '126 ANG MO KIO AVE 3',
 '403 ANG MO KIO AVE 10',
 '404 ANG MO KIO AVE 10',
 '405 ANG MO KIO AVE 10',
 '417 ANG MO KIO AVE 10',
 '418 ANG MO KIO AVE 10',
 '419 ANG MO KIO AVE 10',
 '441 ANG MO KIO AVE 10',
 '442 ANG MO KIO AVE 10',
 '443 ANG MO KIO AVE 10',
 '444 ANG MO KIO AVE 10',
 '450 ANG MO KIO AVE 10',
 '435 ANG MO KIO AVE 10',
 '433 ANG MO KIO AVE 10',
 '434 ANG MO

## Batch Geocoding

Next, we write a simple script to execute batch geocoding, i.e. convert addresses to longitudes and latitudes.

The script will loop through all addresses in ```test_address_list``` and convert it into a query form. Then it calls the OneMap API. The API provides searching of address data for a given search value. It returns search results with both latitude, longitude and x, y coordinates of the searched. Our script here returns a list of query results. 

In [130]:
%%time
import requests

query_results = []
querystring = {}

for x in full_address_list:
    url = "https://developers.onemap.sg/commonapi/search"
    

    querystring.update({"searchVal":x,"returnGeom":"Y","getAddrDetails":"Y","pageNum":"1"})
    #print (querystring['searchVal'])
    
    payload = "------WebKitFormBoundary7MA4YWxkTrZu0gW\r\nContent-Disposition: form-data; name=\"searchVal\"\r\n\r\n\r\n------WebKitFormBoundary7MA4YWxkTrZu0gW\r\nContent-Disposition: form-data; name=\"returnGeom {Y/N}\"\r\n\r\n\r\n------WebKitFormBoundary7MA4YWxkTrZu0gW\r\nContent-Disposition: form-data; name=\"getAddrDetails {Y/N}\"\r\n\r\n\r\n------WebKitFormBoundary7MA4YWxkTrZu0gW\r\nContent-Disposition: form-data; name=\"pageNum\"\r\n\r\n\r\n------WebKitFormBoundary7MA4YWxkTrZu0gW--"
    headers = {
    'content-type': "multipart/form-data; boundary=----WebKitFormBoundary7MA4YWxkTrZu0gW",
    'Cache-Control': "no-cache",
    'Postman-Token': "5f4af03e-410e-4b28-93bc-05006b4de49a"
    }

    response = requests.request("GET", url, data=payload, headers=headers, params=querystring)
    

    query_results.append(response.json())

#print(response.text)
print('Search Done.')

Search Done.
Wall time: 3h 42min 2s


In [132]:
type(query_results)

list

In [133]:
len(query_results)

8838

In [100]:
query_results[1]['results']

[{'ADDRESS': '211 ANG MO KIO AVENUE 3 HDB-ANG MO KIO SINGAPORE 560211',
  'BLK_NO': '211',
  'BUILDING': 'HDB-ANG MO KIO',
  'LATITUDE': '1.36919696561704',
  'LONGITUDE': '103.841666636086',
  'LONGTITUDE': '103.841666636086',
  'POSTAL': '560211',
  'ROAD_NAME': 'ANG MO KIO AVENUE 3',
  'SEARCHVAL': 'HDB-ANG MO KIO',
  'X': '28929.0379298832',
  'Y': '39024.3611866999'}]

In [105]:
type(query_results[1]['results'])

list

In [104]:
df1 = pd.DataFrame.from_dict(query_results[0]['results'])

We proceed to create a dataframe out of results retrieved from the API.

In [134]:
latlong_df = pd.DataFrame()
for entry in range(len(query_results)):
    temp_df = pd.DataFrame.from_dict(query_results[entry]['results'])
    latlong_df = latlong_df.append(temp_df)

In [135]:
latlong_df.head()

,ADDRESS,BLK_NO,BUILDING,LATITUDE,LONGITUDE,LONGTITUDE,POSTAL,ROAD_NAME,SEARCHVAL,X,Y
0,216 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPOR...,216,ANG MO KIO 22,1.36619678831055,103.841505011903,103.841505011903,560216,ANG MO KIO AVENUE 1,ANG MO KIO 22,28911.0522399386,38692.616790574
0,211 ANG MO KIO AVENUE 3 HDB-ANG MO KIO SINGAPO...,211,HDB-ANG MO KIO,1.36919696561704,103.841666636086,103.841666636086,560211,ANG MO KIO AVENUE 3,HDB-ANG MO KIO,28929.0379298832,39024.3611866999
0,202 ANG MO KIO AVENUE 3 ANG MO KIO VIEW SINGAP...,202,ANG MO KIO VIEW,1.3684464401094,103.844516260527,103.844516260527,560202,ANG MO KIO AVENUE 3,ANG MO KIO VIEW,29246.1670971224,38941.3731856439
1,202 ANG MO KIO AVENUE 3 OCBC Ang Mo Kio Ave 3 ...,202,OCBC Ang Mo Kio Ave 3,1.36845752328242,103.844515336938,103.844515336938,560202,ANG MO KIO AVENUE 3,OCBC Ang Mo Kio Ave 3,29246.0643070498,38942.5987060368
0,235 ANG MO KIO AVENUE 3 KEBUN BARU PALM VIEW S...,235,KEBUN BARU PALM VIEW,1.36682360872345,103.83649123351,103.83649123351,560235,ANG MO KIO AVENUE 3,KEBUN BARU PALM VIEW,28353.0784864295,38761.926084429


In [136]:
latlong_df.shape

(11784, 11)

In [316]:
latlong_df["full_address"] = latlong_df["BLK_NO"].map(str) + ' ' + latlong_df["ROAD_NAME"].map(str)

In [317]:
# Do the same for the dataset which contains HDB resale addresses
latlong_df["lat_long"] = latlong_df["LATITUDE"] +' '+ latlong_df["LONGITUDE"]
# convert lat_long column to tuples
latlong_df["lat_long"] = latlong_df["lat_long"].apply(lambda x: tuple(x.split(' ')))

In [319]:
# Save/load latlong_df
latlong_df.to_pickle('data/latlong_df.pkl')    
#latlong_df = pd.read_pickle('data/latlong_df.pkl') 

---

## Distance to Nearest MRT Station Exit

Since we want to build a dashboard that allows users to input their budget and optimize the recommendation of flat types using a variety of considerations. These considerations can be something like, "With my budget, show me the the town that has largest flat type, and also one that is closest to a MRT station."

The following code is on importing *MRT stations' Exits Address* and *Schools' Address* data, conducting geocoding through OneMap API, and calculating the distance of each address to the its nearest station. 

All distances are geodesic distances between two points, adapted from the Python's ```geopy``` package. 

In [308]:
# shapefile to csv converter; 
# Thanks to https://gist.github.com/aerispaha/f098916ac041c286ae92d037ba5c37ba
def read_shapefile(shp_path):
    """
    Read a shapefile into a Pandas dataframe with a 'coords' column holding
    the geometry information. This uses the pyshp package
    """
    import shapefile

    myshp = open(shp_path, "rb")
    #read file, parse out the records and shapes
    sf = shapefile.Reader(shp_path)
    fields = [x[0] for x in sf.fields][1:]
    records = sf.records()
    shps = [s.points for s in sf.shapes()]

    #write into a dataframe
    df = pd.DataFrame(columns=fields, data=records)
    df = df.assign(coords=shps)

    return df

In [309]:
mrt_latlong_df = read_shapefile("data/MRTLRTStnPtt.shp")

In [310]:
station_address_list = mrt_latlong_df["STN_NAME"].unique().tolist()
len(station_address_list)

164

This dataset contains address of 164 MRT stations' exits. So we proceed to get their corresponding longitude and latitude.

In [311]:
%%time
import requests

station_search_results = []
querystring = {}

for x in station_address_list:
    url = "https://developers.onemap.sg/commonapi/search"
    

    querystring.update({"searchVal":x,"returnGeom":"Y","getAddrDetails":"Y","pageNum":"1"})
    #print (querystring['searchVal'])
    
    payload = "------WebKitFormBoundary7MA4YWxkTrZu0gW\r\nContent-Disposition: form-data; name=\"searchVal\"\r\n\r\n\r\n------WebKitFormBoundary7MA4YWxkTrZu0gW\r\nContent-Disposition: form-data; name=\"returnGeom {Y/N}\"\r\n\r\n\r\n------WebKitFormBoundary7MA4YWxkTrZu0gW\r\nContent-Disposition: form-data; name=\"getAddrDetails {Y/N}\"\r\n\r\n\r\n------WebKitFormBoundary7MA4YWxkTrZu0gW\r\nContent-Disposition: form-data; name=\"pageNum\"\r\n\r\n\r\n------WebKitFormBoundary7MA4YWxkTrZu0gW--"
    headers = {
    'content-type': "multipart/form-data; boundary=----WebKitFormBoundary7MA4YWxkTrZu0gW",
    'Cache-Control': "no-cache",
    'Postman-Token': "5f4af03e-410e-4b28-93bc-05006b4de49a"
    }

    response = requests.request("GET", url, data=payload, headers=headers, params=querystring)
    

    station_search_results.append(response.json())

#print(response.text)
print('Search Done.')

Search Done.
Wall time: 4min 1s


In [312]:
station_latlong_df = pd.DataFrame()
for entry in range(len(station_search_results)):
    temp_df = pd.DataFrame.from_dict(station_search_results[entry]['results'])
    station_latlong_df = station_latlong_df.append(temp_df)

In [313]:
station_latlong_df.head(1)

,ADDRESS,BLK_NO,BUILDING,LATITUDE,LONGITUDE,LONGTITUDE,POSTAL,ROAD_NAME,SEARCHVAL,X,Y
0,EUNOS MRT STATION EXIT C SINGAPORE,,EUNOS MRT STATION EXIT C,1.31960866729867,103.903178612002,103.903178612002,NIL,NIL,EUNOS MRT STATION EXIT C,35774.7365172636,33541.2481965665


In [314]:
# Concatenate latitude and longitude columns into one column
station_latlong_df["lat_long"] = station_latlong_df["LATITUDE"] +' '+ station_latlong_df["LONGITUDE"]

# convert lat_long column to tuples
station_latlong_df["lat_long"] = station_latlong_df["lat_long"].apply(lambda x: tuple(x.split(' ')))

In [320]:
station_latlong_df.reset_index()

# convert the lat_long column in station_latlong_df to a list
station_latlong_list = list(station_latlong_df.T.iloc[-1,:])

In [321]:
station_latlong_list[0:3]

[('1.31960866729867', '103.903178612002'),
 ('1.31951727513527', '103.902894137829'),
 ('1.31956254560617', '103.90258207739')]

```station_latlong_list``` is a list of stations in tuples of latitude and longitude. We need it for each address to loop over each coordinate in the list, and caluculate the distance.

In [322]:
%%time
station_distance_df = pd.DataFrame(latlong_df["lat_long"])
for x in station_latlong_list:
    station_distance_df[x]=latlong_df['lat_long'].apply(lambda y: geopy.distance.vincenty(y, x).km)

Wall time: 3min 9s


In [323]:
station_distance_df.shape

(11784, 492)

In [324]:
# get the distance from each address to the nearest station
station_distance_df['distance_to_nearest_station'] = station_distance_df.iloc[:,1:].apply(lambda x: min(x), axis=1)

In [325]:
station_distance_df = station_distance_df.iloc[:,[0,-1]]
station_distance_df.head(1)

,lat_long,distance_to_nearest_station
0,"(1.36619678831055, 103.841505011903)",0.954205


Take a look at the last column, ```distance_to_nearest_station``` is the distance to the nearest MRT station's exit, measured in kilometres.

In [334]:
# Merge our results into the main hdb address dataframe: latlong_df
latlong_df_final = pd.merge(latlong_df, station_distance_df, on='lat_long', how='left')

---

## Distance to Nearest School

We repeat the above steps to get distance to nearest school.

In [24]:
school_address_df = pd.read_csv("data/general-information-of-schools.csv")

In [25]:
school_address_df

,school_name,url_address,address,postal_code,telephone_no,telephone_no_2,fax_no,fax_no_2,email_address,mrt_desc,...,session_code,mainlevel_code,sap_ind,autonomous_ind,gifted_ind,ip_ind,mothertongue1_code,mothertongue2_code,mothertongue3_code,special_sdp_offered
0,NATIONAL JUNIOR COLLEGE,www.nationaljc.moe.edu.sg,37 HILLCREST ROAD,288913,64661144,na,64684535,na,NJC@MOE.EDU.SG,"BOTANIC GARDENS MRT, TAN KAH KEE MRT, SIXTH AV...",...,FULL DAY,MIXED LEVEL,No,No,No,Yes,Chinese,Malay,Tamil,"For the Integrated Programme (Junior High), we..."
1,TEMASEK JUNIOR COLLEGE,http://temasekjc.moe.edu.sg,22 BEDOK SOUTH ROAD,469278,64428066,na,64428762,na,TEMASEK_JC@MOE.EDU.SG,BEDOK MRT,...,FULL DAY,MIXED LEVEL,No,No,No,Yes,Chinese,Malay,Tamil,TJC 6-Year Integrated Programme provides a sea...
2,JURONG JUNIOR COLLEGE,http://www.jurongjc.moe.edu.sg,800 CORPORATION ROAD,649809,65624611,na,65624505,na,jjc@moe.edu.sg,"Lakeside, Boon Lay",...,FULL DAY,JUNIOR COLLEGE,No,No,No,No,Chinese,Malay,Tamil,1. Experiential Learning Programmes (ELP) 2. E...
3,ANDERSON JUNIOR COLLEGE,ajc.moe.edu.sg.,4500 ANG MO KIO AVENUE 6,569843,64596822,na,64598734,na,anderson_jc@moe.edu.sg,Yio Chu Kang,...,FULL DAY,JUNIOR COLLEGE,No,No,No,No,na,na,na,"Science research, service-learning, student le..."
4,VICTORIA JUNIOR COLLEGE,http://www.victoriajc.moe.edu.sg/,20 MARINE VISTA,449035,64485011,na,64438337,na,victoria_jc@moe.edu.sg,"Nearest MRT Stations: Kembangan, Eunos",...,FULL DAY,JUNIOR COLLEGE,No,No,No,Yes,Chinese,Malay,Tamil,Arts Programme Beyond Borders Programme Highe...
5,YISHUN JUNIOR COLLEGE,http://www.yishunjc.moe.edu.sg/,3 YISHUN RING ROAD,768675,62579873,na,62574373,na,YISHUN_JC@MOE.EDU.SG,"Yishun, Khatib",...,FULL DAY,JUNIOR COLLEGE,No,No,No,No,na,na,na,"1. Digital art (includes 3D animation, digital..."
6,TAMPINES JUNIOR COLLEGE,http://www.tpjc.moe.edu.sg/,2 TAMPINES AVENUE 9,529564,67841955,na,67810061,na,tpjc@moe.edu.sg,Tampines East (operational date to be confirmed),...,FULL DAY,JUNIOR COLLEGE,No,No,No,No,na,na,na,"Overseas VIA Project, School Twinning Programm..."
7,SERANGOON JUNIOR COLLEGE,http://www.srjc.moe.edu.sg,1033 UPPER SERANGOON ROAD,534768,62850779,na,62825318,na,srjc@moe.edu.sg,Kovan MRT,...,FULL DAY,JUNIOR COLLEGE,No,No,No,No,na,na,na,Camp ASPIRE:Advanced Student Leadership Progra...
8,PIONEER JUNIOR COLLEGE,http://www.pioneerjc.moe.edu.sg,21 TECK WHYE WALK,688258,65646878,na,67651861,na,PJC@MOE.EDU.SG,Choa Chu Kang,...,FULL DAY,JUNIOR COLLEGE,No,No,No,No,Chinese,na,na,Sports and outdoor electives
9,MERIDIAN JUNIOR COLLEGE,http://meridianjc.moe.edu.sg/,21 PASIR RIS STREET 71,518799,63493660,na,63493667,na,meridian_jc@moe.edu.sg,Pasir Ris MRT Station,...,FULL DAY,JUNIOR COLLEGE,No,No,No,No,Chinese,Malay,Tamil,"Chrystal Programme, Careers & Scholarships Pro..."


In [26]:
full_sch_address_list = school_address_df["address"].unique().tolist()
len(full_sch_address_list)

357

There are 357 schools recorded in the dataset ```school_address_df```.

In [27]:
%%time
import requests

sch_query_results = []
querystring = {}

for x in full_sch_address_list:
    url = "https://developers.onemap.sg/commonapi/search"
    

    querystring.update({"searchVal":x,"returnGeom":"Y","getAddrDetails":"Y","pageNum":"1"})
    #print (querystring['searchVal'])
    
    payload = "------WebKitFormBoundary7MA4YWxkTrZu0gW\r\nContent-Disposition: form-data; name=\"searchVal\"\r\n\r\n\r\n------WebKitFormBoundary7MA4YWxkTrZu0gW\r\nContent-Disposition: form-data; name=\"returnGeom {Y/N}\"\r\n\r\n\r\n------WebKitFormBoundary7MA4YWxkTrZu0gW\r\nContent-Disposition: form-data; name=\"getAddrDetails {Y/N}\"\r\n\r\n\r\n------WebKitFormBoundary7MA4YWxkTrZu0gW\r\nContent-Disposition: form-data; name=\"pageNum\"\r\n\r\n\r\n------WebKitFormBoundary7MA4YWxkTrZu0gW--"
    headers = {
    'content-type': "multipart/form-data; boundary=----WebKitFormBoundary7MA4YWxkTrZu0gW",
    'Cache-Control': "no-cache",
    'Postman-Token': "5f4af03e-410e-4b28-93bc-05006b4de49a"
    }

    response = requests.request("GET", url, data=payload, headers=headers, params=querystring)
    

    sch_query_results.append(response.json())

#print(response.text)
print('Search Done.')

Search Done.
Wall time: 8min 47s


In [29]:
sch_latlong_df = pd.DataFrame()
for entry in range(len(sch_query_results)):
    temp_df = pd.DataFrame.from_dict(sch_query_results[entry]['results'])
    sch_latlong_df = sch_latlong_df.append(temp_df)

In [144]:
sch_latlong_df.head(1)

,ADDRESS,BLK_NO,BUILDING,LATITUDE,LONGITUDE,LONGTITUDE,POSTAL,ROAD_NAME,SEARCHVAL,X,Y
0,37 HILLCREST ROAD NATIONAL JUNIOR COLLEGE SING...,37,NATIONAL JUNIOR COLLEGE,1.33029291072176,103.804110905425,103.804110905425,288913,HILLCREST ROAD,NATIONAL JUNIOR COLLEGE,24749.4869699031,34722.567163026


In [146]:
# Concatenate latitude and longitude columns into one column
sch_latlong_df["lat_long"] = sch_latlong_df["LATITUDE"] +' '+ sch_latlong_df["LONGITUDE"]

# convert lat_long column to tuples
sch_latlong_df["lat_long"] = sch_latlong_df["lat_long"].apply(lambda x: tuple(x.split(' ')))

In [147]:
sch_latlong_df.reset_index()

# convert the lat_long column in station_latlong_df to a list
sch_latlong_list = list(sch_latlong_df.T.iloc[-1,:])

In [148]:
%%time
sch_distance_df = pd.DataFrame(latlong_df["lat_long"])
for x in sch_latlong_list:
    sch_distance_df[x] = latlong_df['lat_long'].apply(lambda y: geopy.distance.vincenty(y, x).km)

Wall time: 3min 44s


In [149]:
# get the distance from each address to the nearest station
sch_distance_df['distance_to_nearest_sch'] = sch_distance_df.iloc[:,1:].apply(lambda x: min(x), axis=1)

In [150]:
sch_distance_df = sch_distance_df.iloc[:,[0,-1]]
sch_distance_df.head(1)

,lat_long,distance_to_nearest_sch
0,"(1.36619678831055, 103.841505011903)",0.145364


In [341]:
# Merge our results into the main hdb address dataframe: latlong_df
latlong_df_final = pd.merge(latlong_df_final, sch_distance_df, on='lat_long', how='left')

In [342]:
latlong_df_final.dtypes

ADDRESS                         object
BLK_NO                          object
BUILDING                        object
LATITUDE                        object
LONGITUDE                       object
LONGTITUDE                      object
POSTAL                          object
ROAD_NAME                       object
SEARCHVAL                       object
X                               object
Y                               object
full_address                    object
lat_long                        object
distance_to_nearest_station    float64
distance_to_nearest_sch        float64
dtype: object

In [343]:
latlong_df_final.head(1)

,ADDRESS,BLK_NO,BUILDING,LATITUDE,LONGITUDE,LONGTITUDE,POSTAL,ROAD_NAME,SEARCHVAL,X,Y,full_address,lat_long,distance_to_nearest_station,distance_to_nearest_sch
0,216 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPOR...,216,ANG MO KIO 22,1.36619678831055,103.841505011903,103.841505011903,560216,ANG MO KIO AVENUE 1,ANG MO KIO 22,28911.0522399386,38692.616790574,216 ANG MO KIO AVENUE 1,"(1.36619678831055, 103.841505011903)",0.954205,0.145364


In [348]:
latlong_df_final = latlong_df_final.iloc[:,[-4,3,4,-1,-2,-3]]

In [349]:
latlong_df_final.head(1)

,full_address,LATITUDE,LONGITUDE,distance_to_nearest_sch,distance_to_nearest_station,lat_long
0,216 ANG MO KIO AVENUE 1,1.36619678831055,103.841505011903,0.145364,0.954205,"(1.36619678831055, 103.841505011903)"


In [448]:
# Save it as csv file
#latlong_df_final.to_csv('data/latlong_df_final.csv')
latlong_df_final = pd.read_csv("data/latlong_df_final.csv")

---

## Extra Information on HDB Blocks

We also want to include some extra information regarding on HDB blocks. This was taken from [data.gov.sg/hdb-property-information](https://data.gov.sg/dataset/hdb-property-information). 

In [186]:
hdb_extra_df = pd.read_csv("data/hdb-property-information.csv")

In [187]:
hdb_extra_df.head(1)

,blk_no,street,max_floor_lvl,year_completed,residential,commercial,market_hawker,miscellaneous,multistorey_carpark,precinct_pavilion,...,3room_sold,4room_sold,5room_sold,exec_sold,multigen_sold,studio_apartment_sold,1room_rental,2room_rental,3room_rental,other_room_rental
0,60,SENG POH LANE,1,1937,N,Y,N,N,N,N,...,0,0,0,0,0,0,0,0,0,0


In [188]:
hdb_extra_df.dtypes

blk_no                   object
street                   object
max_floor_lvl             int64
year_completed            int64
residential              object
commercial               object
market_hawker            object
miscellaneous            object
multistorey_carpark      object
precinct_pavilion        object
bldg_contract_town       object
total_dwelling_units      int64
1room_sold                int64
2room_sold                int64
3room_sold                int64
4room_sold                int64
5room_sold                int64
exec_sold                 int64
multigen_sold             int64
studio_apartment_sold     int64
1room_rental              int64
2room_rental              int64
3room_rental              int64
other_room_rental         int64
dtype: object

Observe that definitely not all features are useful for modeling. 

In [189]:
hdb_extra_df['full_address'] = hdb_extra_df["blk_no"].map(str) + ' ' + hdb_extra_df["street"].map(str)

In [190]:
# Filter out instances that are not commercial buildings
hdb_extra_df = hdb_extra_df[hdb_extra_df["commercial"]=='N']

# Select columns that may be useful in modeling
hdb_extra_df_final = hdb_extra_df.iloc[:,[-1,2,3,4,6,7,8,9]]

In [193]:
hdb_extra_df_final.head(1)

,full_address,max_floor_lvl,year_completed,residential,market_hawker,miscellaneous,multistorey_carpark,precinct_pavilion
8,48 MOH GUAN TER,4,1949,Y,N,Y,N,N


In [194]:
# Save it as csv file
hdb_extra_df_final.to_csv('data/latlong_df_final.csv')

In [296]:
hdb_extra_df_final.shape

(9559, 8)

---

## Final Merge

The data preprocessing stage is often tedious. We should take extra caution when dealing with merging tasks. The column names must match between dataframes, and even entries inside each of them have to be investigated for any spelling errors or typos or mismatchs. 

We found that ```latlong_df_final``` contain addresses in their full form, whereas the concocatenated dataframe ```df``` has address with abbreviations. We have to make sure all full forms strings are rename into their corresponding abbreviations. I figured out the abbreviations by eyeballing the addresses. 

In [451]:
latlong_df_final['full_address'] = latlong_df_final['full_address'].str.replace('AVENUE', 'AVE') \
                                    .str.replace('CRESCENT', 'CRES') \
                                    .str.replace('ROAD', 'RD') \
                                    .str.replace('STREET', 'ST') \
                                    .str.replace('CENTRAL', 'CTRL') \
                                    .str.replace('HEIGHTS', 'HTS') \
                                    .str.replace('TERRACE', 'TER') \
                                    .str.replace('JALAN', 'JLN') \
                                    .str.replace('DRIVE', 'DR') \
                                    .str.replace('PLACE', 'PL') \
                                    .str.replace('CLOSE', 'CL') \
                                    .str.replace('PARK', 'PK') \
                                    .str.replace('GARDENS', 'GDNS') \
                                    .str.replace('NORTH', 'NTH') \
                                    .str.replace('SOUTH', 'STH') \
                                    .str.replace('BUKIT', 'BT') \
                                    .str.replace('UPPER', 'UPP}') \
                                    .str.replace('COMMONWEALTH', "C'WEALTH") \

In [454]:
latlong_df_final_nodup = latlong_df_final.drop_duplicates('full_address')

In [465]:
del latlong_df_final_nodup["Unnamed: 0"]

In [466]:
df_final = pd.merge(df, latlong_df_final_nodup, on='full_address', how='left')

In [471]:
df_final = pd.merge(df_final, hdb_extra_df_final, on='full_address', how='left')

In [473]:
df_final.head(1)

,block,flat_model,flat_type,floor_area_sqm,lease_commence_date,month,resale_price,storey_range,street_name,town,full_address,LATITUDE,LONGITUDE,distance_to_nearest_sch,distance_to_nearest_station,lat_long,max_floor_lvl,year_completed,residential,market_hawker,miscellaneous,multistorey_carpark,precinct_pavilion
0,309,IMPROVED,1 ROOM,31.0,1977,1990-01,9000.0,10 TO 12,ANG MO KIO AVE 1,ANG MO KIO,309 ANG MO KIO AVE 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [461]:
# Thanks to https://stackoverflow.com/a/39734251
def missing_values_table(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns

In [474]:
missing_values_table(df_final)

Your selected dataframe has 23 columns.
There are 12 columns that have missing values.


,Missing Values,% of Total Values
max_floor_lvl,760400,24.5
year_completed,760400,24.5
residential,760400,24.5
market_hawker,760400,24.5
miscellaneous,760400,24.5
multistorey_carpark,760400,24.5
precinct_pavilion,760400,24.5
LATITUDE,156896,5.1
LONGITUDE,156896,5.1
distance_to_nearest_sch,156896,5.1


The above dataframe spells bad news. The new data that we imported, ```hdb_extra_df```, does not have records of 24.5% of address in our original dataframe. I certainly did not expect this. Note that for some of the features, 5.1% of the rows has missing values. Once again this can be attributed to the same reason, ```latlong_df``` does not have records of some addresses in our original dataframe. 

In [481]:
# Create year and month features
df_final['year'] = df_final1['month'].apply(lambda x: x.split('-')[0])
df_final['month'] = df_final1['month'].apply(lambda x: x.split('-')[1])

---

## COE data

In [53]:
coe_df = pd.read_csv("data/coe-results.csv")

In [54]:
coe_df.head()

,month,bidding_no,vehicle_class,quota,bids_success,bids_received,premium
0,2010-01,1,Category A,1152,1145,1342,18502
1,2010-01,1,Category B,687,679,883,19190
2,2010-01,1,Category C,173,173,265,19001
3,2010-01,1,Category D,373,365,509,889
4,2010-01,1,Category E,586,567,1011,19889


In [56]:
coe_df['year'] = coe_df['month'].apply(lambda x: x.split('-')[0])
coe_df['month'] = coe_df['month'].apply(lambda x: x.split('-')[1])

In [63]:
coe_df['month'] = pd.to_numeric(coe_df['month'])

In [73]:
coe_df['year_month'] = coe_df['year'].map(str) + '-' + coe_df['month'].map(str)

In [99]:
coe_prem_df = coe_df.groupby(["year_month"])['premium'].mean().reset_index()

In [100]:
coe_prem_df.head()

,year_month,premium
0,2010-1,16321.3
1,2010-10,30652.9
2,2010-11,32744.5
3,2010-12,43799.2
4,2010-2,18077.2


In [102]:
# Save it as csv file
coe_prem_df.to_csv('data/coe_prem_df.csv')
#df_final = pd.read_csv("data/df_final.csv")

---

## Final Touchups

Some final touchups on our data before we ship it to tableau.

In [485]:
df_final['flat_type'].unique()

array(['1 ROOM', '3 ROOM', '4 ROOM', '5 ROOM', '2 ROOM', 'EXECUTIVE',
       'MULTI GENERATION', 'MULTI-GENERATION'], dtype=object)

In [486]:
df_final['flat_type'] = df_final['flat_type'].str.replace('MULTI-GENERATION', 'MULTI GENERATION')

In [487]:
df_final['flat_type'].unique()

array(['1 ROOM', '3 ROOM', '4 ROOM', '5 ROOM', '2 ROOM', 'EXECUTIVE',
       'MULTI GENERATION'], dtype=object)

In [4]:
df_final.head()

,Unnamed: 0,block,flat_model,flat_type,floor_area_sqm,lease_commence_date,month,resale_price,storey_range,street_name,town,full_address,LATITUDE,LONGITUDE,distance_to_nearest_sch,distance_to_nearest_station,lat_long,max_floor_lvl,year_completed,residential,market_hawker,miscellaneous,multistorey_carpark,precinct_pavilion,year
0,0,309,IMPROVED,1 ROOM,31.0,1977,1,9000.0,10 TO 12,ANG MO KIO AVE 1,ANG MO KIO,309 ANG MO KIO AVE 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1990
1,1,309,IMPROVED,1 ROOM,31.0,1977,1,6000.0,04 TO 06,ANG MO KIO AVE 1,ANG MO KIO,309 ANG MO KIO AVE 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1990
2,2,309,IMPROVED,1 ROOM,31.0,1977,1,8000.0,10 TO 12,ANG MO KIO AVE 1,ANG MO KIO,309 ANG MO KIO AVE 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1990
3,3,309,IMPROVED,1 ROOM,31.0,1977,1,6000.0,07 TO 09,ANG MO KIO AVE 1,ANG MO KIO,309 ANG MO KIO AVE 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1990
4,4,216,NEW GENERATION,3 ROOM,73.0,1976,1,47200.0,04 TO 06,ANG MO KIO AVE 1,ANG MO KIO,216 ANG MO KIO AVE 1,1.366197,103.841505,0.145364,0.954205,"('1.36619678831055', '103.841505011903')",10.0,1975.0,Y,N,N,N,N,1990


In [37]:
df_final.shape

(3100996, 25)

In [38]:
df_final = df_final.dropna(subset=['lat_long'])

In [39]:
df_final["lat_long"] = df_final["LATITUDE"].map(str) +' '+ df_final["LONGITUDE"].map(str)
# convert lat_long column to tuples
df_final["lat_long"] = df_final["lat_long"].apply(lambda x: tuple(x.split(' ')))

In [41]:
%%time
# The coordinates for central area is taken from 
# https://tools.wmflabs.org/geohack/geohack.php?pagename=Central_Region,_Singapore&params=1_18_2.04_N_103_49_17.97_E_type:city(939890)_region:SG

df_final['distance_to_city'] = df_final['lat_long'].apply(lambda y: geopy.distance.vincenty((1.300567, 103.821658), y).km)

Wall time: 1min 24s


In [42]:
df_final['distance_to_city'].describe()

count    2.944100e+06
mean     1.119336e+01
std      3.966534e+00
min      1.089058e+00
25%      8.574658e+00
50%      1.172617e+01
75%      1.448055e+01
max      2.089332e+01
Name: distance_to_city, dtype: float64

## Fixing ```flat_model``` Feature

The feature ```flat_model``` has lots of similar unmatched words and groups that are too small to consider as a categories. We need to rename them one by one and group small groups into one bigger category. We put small groups into a big category mainly because we don't want the dimensionality to be large. If unhandled, it can make the training of models slower, although it does not cause more of a problem in other aspects. 

In [43]:
list(df_final['flat_model'].unique())

['NEW GENERATION',
 'STANDARD',
 'IMPROVED',
 'MODEL A',
 'SIMPLIFIED',
 'MODEL A-MAISONETTE',
 'APARTMENT',
 'MAISONETTE',
 'ADJOINED FLAT',
 'TERRACE',
 '2-ROOM',
 'IMPROVED-MAISONETTE',
 'MULTI GENERATION',
 'PREMIUM APARTMENT',
 'New Generation',
 'Improved',
 'Model A',
 'Standard',
 'Apartment',
 'Model A-Maisonette',
 'Maisonette',
 'Simplified',
 'Multi Generation',
 'Adjoined flat',
 'Premium Apartment',
 'Terrace',
 'Improved-Maisonette',
 'Premium Maisonette',
 '2-room',
 'Model A2',
 'Type S1',
 'Type S2',
 'DBSS',
 'Premium Apartment Loft',
 'Premium Apartment.']

In [44]:
df_final['flat_model'] = df_final['flat_model'].str.replace('Model A-MAISONETTE', 'Maisonette') \
                                    .str.replace('APARTMENT', 'Apartment') \
                                    .str.replace('MAISONETTE', 'Maisonette') \
                                    .str.replace('IMPROVED', 'Improved') \
                                    .str.replace('SIMPLIFIED', 'Simplified') \
                                    .str.replace('MODEL A-MAISONETTE', 'Maisonette') \
                                    .str.replace('Improved-Maisonette', 'Maisonette') \
                                    .str.replace('Premium Maisonette', 'Maisonette') \
                                    .str.replace('ADJOINED FLAT', 'Adjoined flat') \
                                    .str.replace('TERRACE', 'Terrace') \
                                    .str.replace('2-ROOM', '2-room') \
                                    .str.replace('MULTI GENERATION', 'Multi Generation') \
                                    .str.replace('PREMIUM APARTMENT', 'Premium Apartment') \
                                    .str.replace('Premium Apartment.', 'Premium Apartment') \
                                    .str.replace('Premium Apartment Loft', 'Premium Apartment') \
                                    .str.replace('NEW GENERATION', 'New Generation') \
                                    .str.replace('MODEL A', 'Model A') \
                                    .str.replace('Improved-MAISONETTE', 'Maisonette') \
                                    .str.replace('Model A-Maisonette', 'Maisonette') \
                                    .str.replace('Premium ApartmentLoft', 'Premium Apartment')

In [45]:
df_final['flat_model'].value_counts()

Model A              828236
Improved             721208
New Generation       663536
Simplified           207088
Apartment            118832
Premium Apartment    118620
Maisonette           106492
Standard              74412
STANDARD              57132
Model A2              33940
Adjoined flat          6644
Terrace                2444
DBSS                   2300
Multi Generation       1884
Type S1                 796
Type S2                 364
2-room                  148
PREMIUM Apartment        24
Name: flat_model, dtype: int64

We missed some unmatched words. Let's put the 7 smallest groups into a category 'misc'.

In [46]:
df_final['flat_model'] = df_final['flat_model'].str.replace('Model A-MAISONETTE', 'Maisonette') \
                                    .str.replace('PREMIUM Apartment', 'Premium Apartment') \
                                    .str.replace('APARTMENT', 'Apartment') \
                                    .str.replace('MAISONETTE', 'Maisonette') \
                                    .str.replace('2-room', 'misc') \
                                    .str.replace('Type S2', 'misc') \
                                    .str.replace('Type S1', 'misc') \
                                    .str.replace('Multi Generation', 'misc') \
                                    .str.replace('Terrace', 'misc') \
                                    .str.replace('DBSS', 'misc')

In [49]:
df_final['flat_model'].value_counts()

Model A              828236
Improved             721208
New Generation       663536
Simplified           207088
Apartment            118832
Premium Apartment    118644
Maisonette           106492
Standard              74412
STANDARD              57132
Model A2              33940
misc                   7936
Adjoined flat          6644
Name: flat_model, dtype: int64

## Final Look at Dataset

In [50]:
df_final.head(2)

,Unnamed: 0,block,flat_model,flat_type,floor_area_sqm,lease_commence_date,month,resale_price,storey_range,street_name,town,full_address,LATITUDE,LONGITUDE,distance_to_nearest_sch,distance_to_nearest_station,lat_long,max_floor_lvl,year_completed,residential,market_hawker,miscellaneous,multistorey_carpark,precinct_pavilion,year,distance_to_city
4,4,216,New Generation,3 ROOM,73.0,1976,1,47200.0,04 TO 06,ANG MO KIO AVE 1,ANG MO KIO,216 ANG MO KIO AVE 1,1.366197,103.841505,0.145364,0.954205,"(1.36619678831055, 103.841505011903)",10.0,1975.0,Y,N,N,N,N,1990,7.585695
5,5,211,New Generation,3 ROOM,67.0,1977,1,46000.0,01 TO 03,ANG MO KIO AVE 3,ANG MO KIO,211 ANG MO KIO AVE 3,1.369197,103.841667,0.181780,0.867154,"(1.3691969656170402, 103.841666636086)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1990,7.908701


## Saving...

In [104]:
# Save it as csv file
df_final.to_csv('data/df_final.csv')
#df_final = pd.read_csv("data/df_final.csv")

In [71]:
df_final.head()

,Unnamed: 0,block,flat_model,flat_type,floor_area_sqm,lease_commence_date,month,resale_price,storey_range,street_name,town,full_address,LATITUDE,LONGITUDE,distance_to_nearest_sch,distance_to_nearest_station,lat_long,max_floor_lvl,year_completed,residential,market_hawker,miscellaneous,multistorey_carpark,precinct_pavilion,year,distance_to_city,year_month
4,4,216,New Generation,3 ROOM,73.0,1976,1,47200.0,04 TO 06,ANG MO KIO AVE 1,ANG MO KIO,216 ANG MO KIO AVE 1,1.366197,103.841505,0.145364,0.954205,"(1.36619678831055, 103.841505011903)",10.0,1975.0,Y,N,N,N,N,1990,7.585695,1990-1
5,5,211,New Generation,3 ROOM,67.0,1977,1,46000.0,01 TO 03,ANG MO KIO AVE 3,ANG MO KIO,211 ANG MO KIO AVE 3,1.369197,103.841667,0.181780,0.867154,"(1.3691969656170402, 103.841666636086)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1990,7.908701,1990-1
6,6,202,New Generation,3 ROOM,67.0,1977,1,42000.0,07 TO 09,ANG MO KIO AVE 3,ANG MO KIO,202 ANG MO KIO AVE 3,1.368446,103.844516,0.000000,0.560273,"(1.3684464401094, 103.844516260527)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1990,7.925137,1990-1
7,7,235,New Generation,3 ROOM,67.0,1977,1,38000.0,10 TO 12,ANG MO KIO AVE 3,ANG MO KIO,235 ANG MO KIO AVE 3,1.366824,103.836491,0.203720,1.471209,"(1.36682360872345, 103.83649123351)",12.0,1976.0,Y,N,Y,N,N,1990,7.509993,1990-1
8,8,235,New Generation,3 ROOM,67.0,1977,1,40000.0,04 TO 06,ANG MO KIO AVE 3,ANG MO KIO,235 ANG MO KIO AVE 3,1.366824,103.836491,0.203720,1.471209,"(1.36682360872345, 103.83649123351)",12.0,1976.0,Y,N,Y,N,N,1990,7.509993,1990-1


In [70]:
df_final['year_month'] = df_final['year'].map(str) + '-' + df_final['month'].map(str)

In [103]:
df_final.columns

Index(['Unnamed: 0', 'block', 'flat_model', 'flat_type', 'floor_area_sqm',
       'lease_commence_date', 'month', 'resale_price', 'storey_range',
       'street_name', 'town', 'full_address', 'LATITUDE', 'LONGITUDE',
       'distance_to_nearest_sch', 'distance_to_nearest_station', 'lat_long',
       'max_floor_lvl', 'year_completed', 'residential', 'market_hawker',
       'miscellaneous', 'multistorey_carpark', 'precinct_pavilion', 'year',
       'distance_to_city', 'year_month'],
      dtype='object')